# Main generator


In [120]:
import random
from copy import deepcopy
from typing import Literal, Optional

import numpy as np
import pandas as pd
from tqdm import tqdm

In [121]:
# Without alignment
QWERTY_LOW_LAYOUT_: list[list[str]] = [
    ["`", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "-", "=", "<back>"],
    ["<tab>", "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "[", "]", "\\"],
    ["<caps>", "a", "s", "d", "f", "g", "h", "j", "k", "l", ";", "'", "<enter>"],
    ["<shift>", "z", "x", "c", "v", "b", "n", "m", ",", ".", "/", "<shift>"],
    [
        "<ctrl>",
        "<alt>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<alt>",
        "<ctrl>",
    ],
]

QWERTY_HIGH_LAYOUT_: list[list[str]] = [
    ["~", "!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "_", "+", "<back>"],
    ["<tab>", "Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P", "{", "}", "|"],
    ["<caps>", "A", "S", "D", "F", "G", "H", "J", "K", "L", ":", '"', "<enter>"],
    ["<shift>", "Z", "X", "C", "V", "B", "N", "M", "<", ">", "?", "<shift>"],
    [
        "<ctrl>",
        "<alt>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<alt>",
        "<ctrl>",
    ],
]

In [122]:
QWERTY_LOW_LAYOUT: list[list[str]] = [
    ["`", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "-", "=", "<back>"],
    ["<tab>", "q", "w", "e", "r", "t", "y", "u", "i", "o", "p", "[", "]", "\\"],
    [
        "<caps>",
        "a",
        "s",
        "d",
        "f",
        "g",
        "h",
        "j",
        "k",
        "l",
        ";",
        "'",
        "<enter>",
        "<enter>",
    ],
    [
        "<shift>",
        "<shift>",
        "z",
        "x",
        "c",
        "v",
        "b",
        "n",
        "m",
        ",",
        ".",
        "/",
        "<shift>",
        "<shift>",
    ],
    [
        "<ctrl>",
        "<alt>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<alt>",
        "<ctrl>",
    ],
]

QWERTY_HIGH_LAYOUT: list[list[str]] = [
    ["~", "!", "@", "#", "$", "%", "^", "&", "*", "(", ")", "_", "+", "<back>"],
    ["<tab>", "Q", "W", "E", "R", "T", "Y", "U", "I", "O", "P", "{", "}", "|"],
    [
        "<caps>",
        "A",
        "S",
        "D",
        "F",
        "G",
        "H",
        "J",
        "K",
        "L",
        ":",
        '"',
        "<enter>",
        "<enter>",
    ],
    [
        "<shift>",
        "<shift>",
        "Z",
        "X",
        "C",
        "V",
        "B",
        "N",
        "M",
        "<",
        ">",
        "?",
        "<shift>",
        "<shift>",
    ],
    [
        "<ctrl>",
        "<alt>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<space>",
        "<alt>",
        "<ctrl>",
    ],
]

In [123]:
def get_buttons_set(
    low_layout: list[list[str]], high_layout: list[list[str]]
) -> set[str]:
    buttons: set[str] = set()

    for layout in [low_layout, high_layout]:
        for i in range(len(layout)):
            for btn in layout[i]:
                buttons.add(btn)

    return buttons


def get_keyboard_shape(layout: list[list[str]]) -> tuple[int, ...]:
    shape = [len(row) for row in layout]

    return tuple(shape)


BUTTONS_SET = get_buttons_set(QWERTY_LOW_LAYOUT, QWERTY_HIGH_LAYOUT)
KEYBOARD_LAYOUT_SHAPE = get_keyboard_shape(QWERTY_LOW_LAYOUT)

In [124]:
def encode_decode_buttons(buttons: set[str]) -> tuple[dict[str, int], dict[int, str]]:
    letters_dict = {}
    for idx, letter in enumerate("abcdefghijklmnopqrsuvwxyz"):
        letters_dict[letter] = idx + 1
        letters_dict[letter.upper()] = -(idx + 1)

    encode_value = (len(letters_dict) // 2) + 1
    encode_dict = {}
    decode_dict = {}
    for btn in buttons:
        if btn in letters_dict:
            decode_dict[letters_dict[btn]] = btn
            encode_dict[btn] = letters_dict[btn]
        else:
            decode_dict[encode_value] = btn
            encode_dict[btn] = encode_value
            encode_value += 1
    return encode_dict, decode_dict


ENCODE_DICT, DECODE_DICT = encode_decode_buttons(BUTTONS_SET)
ENCODED_BUTTONS_SET = {ENCODE_DICT[btn] for btn in BUTTONS_SET}
SHIFT_CODE = ENCODE_DICT["<shift>"]

In [125]:
for btn in BUTTONS_SET:
    assert btn == DECODE_DICT[ENCODE_DICT[btn]]

In [126]:
Layout = list[list[int]]


def encode_layout(layout: list[list[str]]) -> Layout:
    return [[ENCODE_DICT[btn] for btn in layout[i]] for i in range(len(layout))]


def decode_layout(layout: Layout) -> list[list[str]]:
    return [[DECODE_DICT[btn] for btn in layout[i]] for i in range(len(layout))]


QWERTY_ENCODED_HIGH: Layout = encode_layout(QWERTY_HIGH_LAYOUT)
QWERTY_ENCODED_LOW: Layout = encode_layout(QWERTY_LOW_LAYOUT)

In [127]:
def get_all_buttons_encoded(high_layout: Layout, low_layout: Layout) -> list[int]:
    all_buttons = []

    for layout in (low_layout, high_layout):
        for row in layout:
            all_buttons.extend(row)
    return all_buttons


ALL_BUTTONS_ENCODED = get_all_buttons_encoded(QWERTY_ENCODED_HIGH, QWERTY_ENCODED_LOW)

In [128]:
LogType = Literal["basic"] | Literal["debug"] | Literal["error"]


class Logger:
    def __init__(self, verbose: bool = True, hide_types: list[LogType] = []) -> None:
        self.verbose = verbose
        self.hide_types = set(hide_types)

    def log(self, message: str, log_type: LogType = "basic") -> None:
        if self.verbose and log_type not in self.hide_types:
            print(message)


LOGGER = Logger()

In [129]:
Position = tuple[int, int]


class Finger:
    def __init__(
        self, initial_position: Position, name: str, logger: Logger = LOGGER
    ) -> None:
        self.name = name
        self.initial_position = initial_position

        self.logger = logger

        self.reset()

        # Constants

        self.wait_before_return = 4  # in ticks

        self.long_row_move_shift = 3
        self.long_row_move_penalty = 1

        self.row_penalty_coefficient = 1
        self.column_penalty_coefficient = 1.2

    def reset(self):
        self.current_position = self.initial_position
        self.ticks_before_return = 0  # if == 0, returns to the initial position
        self.typed_keys = 0

    def move(self, position: Position):
        self.current_position = position

        self.ticks_before_return = self.wait_before_return
        self.typed_keys += 1

    def tick(self) -> float:
        if self.ticks_before_return > 0:
            self.ticks_before_return -= 1

        if self.ticks_before_return == 0:
            score = self.get_score(self.initial_position)
            self.current_position = self.initial_position
            return score

        return 0

    def get_score(self, target_position: Position) -> float:
        x1, y1 = self.current_position
        x2, y2 = target_position

        row_distance = abs(x1 - x2) ** 2
        column_distance = abs(y1 - y2) ** 2

        penalty = 0
        if row_distance > self.long_row_move_shift:
            penalty = self.long_row_move_penalty
        return (
            row_distance * self.row_penalty_coefficient
            + column_distance * self.column_penalty_coefficient
            + penalty
        )

    def show_statistics(self):
        self.logger.log(
            f"Name: {self.name:22} \
            Typed keys: {self.typed_keys:5} \
            Ticks before return: {self.ticks_before_return:5} \
            Current position: {self.current_position}\t\
            Default position: {self.initial_position}"
        )

In [130]:
DEFAULT_FINGERS: list[Finger] = [
    Finger((2, 1), "левый мизинец"),
    Finger((2, 2), "левый безымянный"),
    Finger((2, 3), "левый средний"),
    Finger((2, 4), "левый указательный"),
    Finger((4, 3), "левый большой"),
    Finger((4, 6), "правый большой"),
    Finger((2, 7), "правый указательный"),
    Finger((2, 8), "правый средний"),
    Finger((2, 9), "правый безымянный"),
    Finger((2, 10), "правый мизинец"),
]

SwapType = Literal["low_layout"] | Literal["high_layout"] | Literal["between_layouts"]


class KeyboardLayout:
    @staticmethod
    def layout_to_dict(layout: Layout) -> dict[int, list[Position]]:
        layout_dict: dict[int, list[Position]] = {}

        for i in range(len(layout)):
            for j in range(len(layout[i])):
                button = layout[i][j]
                if button in layout_dict:
                    layout_dict[button].append((i, j))
                else:
                    layout_dict[button] = [(i, j)]

        return layout_dict

    def _finish_move(self):
        for finger in self.fingers:
            self.total_score += finger.tick()

    def __init__(self, low_layout: Layout, high_layout: Layout, logger: Logger = LOGGER):
        self.low_layout = deepcopy(low_layout)
        self.high_layout = deepcopy(high_layout)

        self.low_layout_dict = KeyboardLayout.layout_to_dict(self.low_layout)
        self.high_layout_dict = KeyboardLayout.layout_to_dict(self.high_layout)

        self.logger = logger

        self.fingers = deepcopy(DEFAULT_FINGERS)

        self.reset()

    def reset(self):
        self.total_score: float = 0
        for f in self.fingers:
            f.reset()

    def move_one_finger(
        self, positions: list[Position], busy_finger_id: Optional[int] = None
    ) -> tuple[tuple[int, Position], float]:
        best_finger_id: int = 0
        best_score = np.inf

        final_position: Position = (0, 0)

        for position in positions:
            scores = [
                finger.get_score(position) if i != busy_finger_id else np.inf
                for i, finger in enumerate(self.fingers)
            ]

            candidate_finger_id = int(np.argmin(scores))
            candidate_score = scores[candidate_finger_id]

            if candidate_score < best_score:
                best_score = candidate_score
                best_finger_id = candidate_finger_id
                final_position = position

        return (best_finger_id, final_position), best_score

    def move_two_fingers(
        self, positions: list[Position]
    ) -> tuple[tuple[int, Position], tuple[int, Position], float]:
        shift_positions = self.low_layout_dict[SHIFT_CODE]

        # firstly reach SHIFT, then - positions
        finger_shift_info_1, shift_distance_1 = self.move_one_finger(shift_positions)
        finger_btn_info_1, d1_btn = self.move_one_finger(
            positions, finger_shift_info_1[0]
        )
        total_distance_1 = shift_distance_1 + d1_btn

        # firstly reach positions, then - SHIFT
        finger_btn_info_2, d1_btn = self.move_one_finger(positions)
        finger_shift_info_2, shift_distance_2 = self.move_one_finger(
            shift_positions, finger_btn_info_2[0]
        )
        total_distance_2 = shift_distance_2 + d1_btn

        if total_distance_1 < total_distance_2:
            return finger_btn_info_1, finger_shift_info_1, total_distance_1

        return finger_btn_info_2, finger_shift_info_2, total_distance_2

    def find_button(self, button: int):
        if button in self.low_layout_dict:
            (finger_id, finger_position), score = self.move_one_finger(
                self.low_layout_dict[button]
            )

            self.fingers[finger_id].move(finger_position)
            self.total_score += score

            self.logger.log(f"{button}:\t{self.fingers[finger_id].name}")

        elif button in self.high_layout_dict:
            (
                (finger_id_1, finger_position_1),
                (finger_id_2, finger_position_2),
                score,
            ) = self.move_two_fingers(self.high_layout_dict[button])

            self.fingers[finger_id_1].move(finger_position_1)
            self.fingers[finger_id_2].move(finger_position_2)
            self.total_score += score

            self.logger.log(
                f"{button}:\t{self.fingers[finger_id_1].name} + {self.fingers[finger_id_2].name}"
            )

        else:
            self.logger.log(f"NO SUCH KEY: {button}")

        self._finish_move()

    def type_text(self, text: list[str]) -> float:
        for button in text:
            self.find_button(ENCODE_DICT[button])

        return self.total_score

    def type_encoded_text(self, encoded_text: list[int]) -> float:
        for button in encoded_text:
            self.find_button(button)
        return self.total_score

    def swap_buttons(self, position1: Position, position2: Position, swap_type: SwapType):
        if swap_type == "high_layout":
            layout_from = layout_to = self.high_layout
            layout_from_dict = layout_to_dict = self.high_layout_dict
        elif swap_type == "low_layout":
            layout_from = layout_to = self.low_layout
            layout_from_dict = layout_to_dict = self.low_layout_dict
        else:  # swap_type == "between_layouts"
            layout_from = self.low_layout
            layout_to = self.high_layout
            layout_from_dict = self.low_layout_dict
            layout_to_dict = self.high_layout_dict

        x1, y1 = position1
        btn1 = layout_from[x1][y1]
        x2, y2 = position2
        btn2 = layout_to[x2][y2]

        layout_from[x1][y1], layout_to[x2][y2] = layout_to[x2][y2], layout_from[x1][y1]

        layout_from_dict[btn1].remove(position1)
        layout_from_dict[btn1].append(position2)

        layout_to_dict[btn2].append(position1)
        layout_to_dict[btn2].remove(position2)

    def decode_layouts(self) -> tuple[list[list[str]], list[list[str]]]:
        return (decode_layout(self.low_layout), decode_layout(self.high_layout))

    def get_sting_layouts(self) -> str:
        low_layout, high_layout = self.decode_layouts()
        result_string = "High layout:\n"
        for row in high_layout:
            for s in row:
                result_string += f"{s:8}"
            result_string += "\n"
        result_string += "\n"

        result_string += "\nLow layout:\n"
        for row in low_layout:
            for s in row:
                result_string += f"{s:8}"
            result_string += "\n"
        result_string += "\n"

        return result_string

    def show_statistics(self):
        self.logger.log("\nStatistics:")
        for f in self.fingers:
            f.show_statistics()

In [131]:
def generate_random_layouts(
    all_buttons_encoded: list[int],
    keyboard_shape: tuple[int, ...],
    seed: Optional[int] = None,
) -> tuple[Layout, Layout]:
    if seed is not None:
        random.seed(seed)

    all_buttons = all_buttons_encoded.copy()

    low_layout = [[0 for _ in range(row_length)] for row_length in keyboard_shape]
    high_layout = [[0 for _ in range(row_length)] for row_length in keyboard_shape]

    # Push single SHIFT to the low layout
    all_buttons.remove(SHIFT_CODE)
    random.shuffle(all_buttons)

    shift_position_low_layout = random.randint(0, len(all_buttons_encoded) // 2)
    all_buttons.insert(shift_position_low_layout, SHIFT_CODE)
    pointer = 0
    for layout in (low_layout, high_layout):
        for i, row_length in enumerate(keyboard_shape):
            for j in range(row_length):
                layout[i][j] = all_buttons[pointer]
                pointer += 1

    return low_layout, high_layout


sample_low_layout, sample_high_layout = generate_random_layouts(
    ALL_BUTTONS_ENCODED, KEYBOARD_LAYOUT_SHAPE
)
sample_low_layout, sample_high_layout

([[41, 45, 65, -13, 51, 67, 36, 37, 3, 27, 67, 40, 56, 1],
  [39, 41, 67, 30, -2, 16, 32, 4, 31, -16, 52, 33, -7, -12],
  [11, 5, 18, -23, 20, 61, 74, 17, 41, -21, 66, 67, 56, 47],
  [-11, 57, 67, 23, 43, 67, -9, 72, -24, 54, 22, 56, 10, 30],
  [-5, 30, 68, 67, 29, 64, 48, -18, 30, 73, 50]],
 [[63, 34, 28, 19, 67, 21, -8, 26, 67, 41, -1, -14, 60, -4],
  [2, 70, 71, 42, -15, -17, 58, 65, 31, 53, 9, 65, 7, 77],
  [30, 38, 67, 67, 56, 12, 30, 25, -22, -6, 8, -19, 62, -3],
  [75, 76, 65, 35, -25, 55, 6, 44, 67, 30, 14, 13, 69, 49],
  [30, -20, -10, 24, 75, 59, 77, 46, 15, 67, 67]])

### Test Keyboard

In [132]:
qwerty_keyboard = KeyboardLayout(QWERTY_ENCODED_LOW, QWERTY_ENCODED_HIGH, LOGGER)
print(qwerty_keyboard.get_sting_layouts())

High layout:
~       !       @       #       $       %       ^       &       *       (       )       _       +       <back>  
<tab>   Q       W       E       R       T       Y       U       I       O       P       {       }       |       
<caps>  A       S       D       F       G       H       J       K       L       :       "       <enter> <enter> 
<shift> <shift> Z       X       C       V       B       N       M       <       >       ?       <shift> <shift> 
<ctrl>  <alt>   <space> <space> <space> <space> <space> <space> <space> <alt>   <ctrl>  


Low layout:
`       1       2       3       4       5       6       7       8       9       0       -       =       <back>  
<tab>   q       w       e       r       t       y       u       i       o       p       [       ]       \       
<caps>  a       s       d       f       g       h       j       k       l       ;       '       <enter> <enter> 
<shift> <shift> z       x       c       v       b       n       m       ,       .       /    

In [133]:
print(qwerty_keyboard.low_layout_dict[ENCODE_DICT["s"]])
print(qwerty_keyboard.low_layout_dict[ENCODE_DICT["2"]])
print(qwerty_keyboard.high_layout_dict[ENCODE_DICT["S"]])
print(qwerty_keyboard.high_layout_dict[ENCODE_DICT["@"]])

[(2, 2)]
[(0, 2)]
[(2, 2)]
[(0, 2)]


In [134]:
qwerty_keyboard.swap_buttons((2, 2), (0, 2), "low_layout")
qwerty_keyboard.swap_buttons((2, 2), (0, 2), "high_layout")

print(qwerty_keyboard.low_layout_dict[ENCODE_DICT["s"]])
print(qwerty_keyboard.low_layout_dict[ENCODE_DICT["2"]])
print(qwerty_keyboard.high_layout_dict[ENCODE_DICT["S"]])
print(qwerty_keyboard.high_layout_dict[ENCODE_DICT["@"]])

[(0, 2)]
[(2, 2)]
[(0, 2)]
[(2, 2)]


In [135]:
print(qwerty_keyboard.low_layout[0][5])
print(qwerty_keyboard.high_layout[1][6])

51
-24


In [136]:
qwerty_keyboard.swap_buttons((0, 5), (1, 6), "between_layouts")
print(qwerty_keyboard.low_layout[0][5])
print(qwerty_keyboard.high_layout[1][6])

-24
51


In [137]:
qwerty_keyboard.show_statistics()


Statistics:
Name: левый мизинец                      Typed keys:     0             Ticks before return:     0             Current position: (2, 1)	            Default position: (2, 1)
Name: левый безымянный                   Typed keys:     0             Ticks before return:     0             Current position: (2, 2)	            Default position: (2, 2)
Name: левый средний                      Typed keys:     0             Ticks before return:     0             Current position: (2, 3)	            Default position: (2, 3)
Name: левый указательный                 Typed keys:     0             Ticks before return:     0             Current position: (2, 4)	            Default position: (2, 4)
Name: левый большой                      Typed keys:     0             Ticks before return:     0             Current position: (4, 3)	            Default position: (4, 3)
Name: правый большой                     Typed keys:     0             Ticks before return:     0             Current position:

In [138]:
qwerty_keyboard.type_text(list("Procrastination!"))

-16:	правый мизинец + левый мизинец
18:	левый указательный
15:	правый безымянный
3:	левый средний
18:	левый указательный
1:	левый мизинец
19:	левый безымянный
70:	левый указательный
9:	правый средний
14:	правый указательный
1:	левый мизинец
70:	левый указательный
9:	правый средний
15:	правый безымянный
14:	правый указательный
44:	левый мизинец + левый безымянный


44.400000000000006

In [139]:
qwerty_keyboard.show_statistics()


Statistics:
Name: левый мизинец                      Typed keys:     4             Ticks before return:     3             Current position: (0, 1)	            Default position: (2, 1)
Name: левый безымянный                   Typed keys:     2             Ticks before return:     3             Current position: (3, 1)	            Default position: (2, 2)
Name: левый средний                      Typed keys:     1             Ticks before return:     0             Current position: (2, 3)	            Default position: (2, 3)
Name: левый указательный                 Typed keys:     4             Ticks before return:     0             Current position: (2, 4)	            Default position: (2, 4)
Name: левый большой                      Typed keys:     0             Ticks before return:     0             Current position: (4, 3)	            Default position: (4, 3)
Name: правый большой                     Typed keys:     0             Ticks before return:     0             Current position:

As we can see, we successfully can swap different keys in given keyboard and calculate distance function fro given string.


## Data loading & preprocessing

In [27]:
data_frame = pd.read_csv("../data/raw/cpp_programs.csv", index_col=0)
data_frame.head()

,text
0,"point operator-(point p1,point p2)\n p1.x-=p2..."
1,"int main() {\n map<char, int> m;\n m['6'..."
2,"int main()\n\tint n,s[1001],cnt=0;\n\tscanf(""%..."
3,int main() {\n\tcin >> n;\n\tfor (int i = 0; i...
4,"int main()\n char a, b, c, d, kozir;\n ..."


In [28]:
text_lengths = data_frame["text"].apply(len)
print(f"Mean length: {text_lengths.mean()}")
print(f"Max length: {text_lengths.max()}")
print(f"Min length: {text_lengths.min()}")

Mean length: 466.05414663010777
Max length: 62313
Min length: 12


In [101]:
MAX_LENGTH = 400
PADDING_VALUE = ENCODE_DICT["<space>"]

SPECIAL_SYMBOLS = {
    "\t": "<tab>",
    "\n": "<enter>",
    " ": "<space>",
}


def preprocess_text(text: str) -> list[int]:
    splitted_text = [
        SPECIAL_SYMBOLS[s] if s in SPECIAL_SYMBOLS else s for s in list(text)
    ]
    encoded_text = [
        ENCODE_DICT[symbol]
        for symbol in list(filter(lambda s: s in BUTTONS_SET, splitted_text))[:MAX_LENGTH]
    ]
    return encoded_text + [PADDING_VALUE for _ in range(MAX_LENGTH - len(encoded_text))]


data_frame["encoded_text"] = data_frame["text"].apply(preprocess_text)

In [102]:
dataset = np.array([np.array(text) for text in data_frame["encoded_text"].to_list()])
dataset.shape

(119989, 400)

## Measure QWERTY

In [140]:
qwerty_keyboard = KeyboardLayout(
    QWERTY_ENCODED_LOW, QWERTY_ENCODED_HIGH, Logger(verbose=False)
)

random_low, random_high = generate_random_layouts(
    ALL_BUTTONS_ENCODED, KEYBOARD_LAYOUT_SHAPE, seed=42
)
random_keyboard = KeyboardLayout(random_low, random_high, Logger(verbose=False))


loop = tqdm(dataset[:100])
for text in loop:
    qwerty_score_total = qwerty_keyboard.type_encoded_text(text)
    random_score_total = random_keyboard.type_encoded_text(text)
    loop.set_postfix({"qwerty": qwerty_score_total, "random": random_score_total})

100%|██████████| 100/100 [00:05<00:00, 19.15it/s, qwerty=1e+5, random=1.23e+5]  


In [141]:
print(f"{qwerty_keyboard.total_score=:.0f}")
print(f"{random_keyboard.total_score=:.0f}")

qwerty_keyboard.total_score=100223
random_keyboard.total_score=123039


In [142]:
print(random_keyboard.get_sting_layouts())

High layout:
H       &       V       <tab>   8       p       v       l       <space> 0       <space> q       @       <space> 
5       g       X       x       c       =       n       N       <space> |       <ctrl>  t       G       z       
>       "       y       D       `       <space> ?       <space> <ctrl>  <shift> <shift> O       E       {       
[       %       M       _       <alt>   a       \       <shift> 3       4       <shift> (       -       #       
B       <space> Y       <back>  <caps>  j       <space> <space> $       6       <caps>  


Low layout:
]       m       F       b       <enter> u       1       :       d       ^       9       w       <alt>   '       
<shift> U       7       P       s       <space> .       <alt>   C       ,       f       K       <space> <ctrl>  
r       <alt>   Q       *       <shift> k       2       !       Z       S       <tab>   }       /       <shift> 
<space> R       W       h       J       <space> ;       <       <enter> A       <enter> <spac